In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 7-3. ブロードキャスティングの適用

### 実例(1) 配列の派生型

#### コード7-23. `GeometricSequence.jl`（1～18行目、コード6-3(1). 再掲）

```julia
struct GeometricSequence{T<:Number} <: AbstractVector{T}
    a::T
    r::T
    n::Int
end

function GeometricSequence(a::T1, r::T2, n::Integer) where {T1<:Number, T2<:Number}
    GeometricSequence(promote(a, r)..., Int(n))
end

Base.length(seq::GeometricSequence) = seq.n
Base.size(seq::GeometricSequence) = (seq.n,)
Base.getindex(seq::GeometricSequence, index::Integer) = seq.a * seq.r ^ (index - 1)

function Base.show(io::IO, seq::GeometricSequence)
    print(io, "GeometricSequence(", seq.a, ", ", seq.r, ", ", seq.n, ")")
end
Base.show(io::IO, ::MIME"text/plain", seq::GeometricSequence) = show(io, seq)
```

In [2]:
include("GeometricSequence.jl")

#### コード7-24. `GeometricSequence` のブロードキャスティング適用確認

In [3]:
seq0 = GeometricSequence(1, 3, 8)

GeometricSequence(1, 3, 8)

In [4]:
seq0 isa AbstractVector

true

In [5]:
Broadcast.broadcastable(seq0)  # return `seq0` itself

GeometricSequence(1, 3, 8)

In [6]:
Broadcast.BroadcastStyle(typeof(seq0))

Base.Broadcast.DefaultArrayStyle{1}()

In [7]:
log.(seq0)

8-element Vector{Float64}:
 0.0
 1.0986122886681098
 2.1972245773362196
 3.295836866004329
 4.394449154672439
 5.493061443340548
 6.591673732008658
 7.690286020676768

In [8]:
seq0 .+ 1  # same as [v + 1 for v in seq0]

8-element Vector{Int64}:
    2
    4
   10
   28
   82
  244
  730
 2188

In [9]:
[1.0 3.0^8] .* seq0

8×2 Matrix{Float64}:
    1.0    6561.0
    3.0   19683.0
    9.0   59049.0
   27.0  177147.0
   81.0  531441.0
  243.0       1.59432e6
  729.0       4.78297e6
 2187.0       1.43489e7

### 実例(2) イテレーションプロトコルを実装した型

#### コード7-25. `Collatz.jl`（コード6-22. 再掲）

```julia
struct Collatz{I <: Integer}
    start::I
end

Base.IteratorSize(::Type{<:Collatz}) = Base.SizeUnknown()

Base.IteratorEltype(::Type{<:Collatz}) = Base.HasEltype()
Base.eltype(::Type{Collatz{I}}) where I = I

Base.iterate(c::Collatz) = (c.start, c.start)
function Base.iterate(::Collatz, prev)
    prev == 1 && return nothing
    next = if iseven(prev)
        prev ÷ 2
    else
        3prev + 1
    end
    (next, next)
end
```

In [10]:
include("Collatz.jl")

#### コード7-26. `Collatz` のブロードキャスティング適用確認

In [11]:
c3 = Collatz(3)

Collatz{Int64}(3)

In [12]:
c3 isa AbstractArray

false

In [13]:
Broadcast.broadcastable(c3)  # equivalent to `collect(c3)`

8-element Vector{Int64}:
  3
 10
  5
 16
  8
  4
  2
  1

In [14]:
Broadcast.BroadcastStyle(typeof(Broadcast.broadcastable(c3)))

Base.Broadcast.DefaultArrayStyle{1}()

In [15]:
seq0 = GeometricSequence(1, 3, 8);

In [16]:
.-c3

8-element Vector{Int64}:
  -3
 -10
  -5
 -16
  -8
  -4
  -2
  -1

In [17]:
max.(c3, seq0)

8-element Vector{Int64}:
    3
   10
    9
   27
   81
  243
  729
 2187

In [18]:
c3 .+ seq0

8-element Vector{Int64}:
    4
   13
   14
   43
   89
  247
  731
 2188